<!-- dom:TITLE: Examples of algorithmic thinking: Numerical Integration -->
# Examples of algorithmic thinking: Numerical Integration
**Aksel Hiorth**
University of Stavanger

Date: **Aug 14, 2023**

<!-- Common Mako variables and functions -->

# Algorithmic thinking

The only way to improve in coding and algorithmic thinking is practice. The concept of one dimensional numerical integration is easy to understand, i.e. to calculate the area under a curve. In this chapter we will implement several numerical methods, and it will serve as a very simple playground that illustrates the key aspects of numerical modeling

1. We start with a mathematical model (in this case an integral)

2. The mathematical model is formulated in discrete form 

3. Then we design an algorithm to solve the model 

4. The numerical solution for a test case is compared with the true solution (could be an analytical solution or data)

5. Error analysis: we investigate the accuracy of the algorithm by changing the number of iterations and/or make changes to the implementation or algorithm

The main point of this chapter is not to develop your own integration methods, the built in methods in Scipy will work in most cases. However, the way to break down the main task of calculating an integral into smaller tasks that is understandable by a computer, may work as a template for many different problems you would typically solve using a computer. A second motivation is that by analyzing the origin of numerical errors gives ideas for improving the algorithm, which is transferable to other problems.      

## The midpoint rule
Numerical integration is encountered in numerous applications in physics and engineering sciences. 
Let us first consider the most simple case, a function $f(x)$, which is a function of one variable, $x$. The most straight forward way of calculating the area $\int_a^bf(x)dx$ is 
simply to divide the area under the function into $N$ equal rectangular slices with size $h=(b-a)/N$, as illustrated in [figure 1](#fig:numint:mid). The area of one box is:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:mid0"></div>

$$
\begin{equation}
M(x_k,x_k+h)=f(x_k+\frac{h}{2}) h,\label{eq:numint:mid0} \tag{1}
\end{equation}
$$

and the area of all the boxes is:

$$
I(a,b)=\int_a^bf(x)dx\simeq\sum_{k=0}^{N-1}M(x_k,x_k+h)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:numint:mid1"></div>

$$
\begin{equation}  
=h\sum_{k=0}^{N-1}f(x_k+\frac{h}{2})=h\sum_{k=0}^{N-1}f(a+(k+\frac{1}{2})h).
\label{eq:numint:mid1} \tag{2}
\end{equation}
$$

Note that the sum goes from $k=0,1,\ldots,N-1$, a total of $N$ elements. We could have chosen to let the sum go from $k=1,2,\ldots,N$. 
In Python, C, C++ and many other programming languages the arrays start by indexing the elements from $0,1,\ldots$ to $N-1$, 
therefore we choose the convention of having the first element to start at $k=0$.

<!-- dom:FIGURE: [fig-numint/func_sq.png, width=800] Integrating a function with the midpoint rule. <div id="fig:numint:mid"></div> -->
<!-- begin figure -->
<div id="fig:numint:mid"></div>

<img src="fig-numint/func_sq.png" width=800><p style="font-size: 0.9em"><i>Figure 1: Integrating a function with the midpoint rule.</i></p>
<!-- end figure -->

Below is a Python code, where this algorithm is implemented for $\int_0^\pi\sin (x)dx$

In [1]:
import numpy as np
# Function to be integrated
def f(x):
    return np.sin(x)

def midpoint(f,a,b,N):
    """
    f : function to be integrated on the domain [a,b]
    N : number of integration points
    """
    h=(b-a)/N
    x=np.arange(a+0.5*h,b,h)
    return h*np.sum(f(x))
N=10
a=0
b=np.pi
Area = midpoint(f,a,b,N)
print('Numerical value= ', Area)
print('Error= ', (2-Area)) # Analytical result is 2

**Notice.**

In the implementation above, we have taken advantage of Numpys ability to pass a vector to a function. This greatly enhances the speed and makes clean, readable code. If you were coding in a lower level programming language like Fortran, C or C++, you would probably implement the loop like (in Python syntax):

In [2]:
for k in range(0,N): # loop over k=0,1,..,N-1
    val = lower_limit+(k+0.5)*h # midpoint value
    area += func(val)
return area*h

By increasing $N$ the numerical result will get closer to the true answer. How much do you need to increase $N$ in order to reach an accuracy higher than $10^{-8}$.
 What happens when $N$ increases?

## The trapezoidal rule
The numerical error in the above example is quite low, only about 2$\%$ for $N=5$. 
However, by just looking at the graph above it seems likely that we can develop a better algorithm by using trapezoids instead of rectangles, 
see [figure 2](#fig:numint:trap).

<!-- dom:FIGURE: [fig-numint/func_tr.png, width=800] Integrating a function with the trapezoidal rule. <div id="fig:numint:trap"></div> -->
<!-- begin figure -->
<div id="fig:numint:trap"></div>

<img src="fig-numint/func_tr.png" width=800><p style="font-size: 0.9em"><i>Figure 2: Integrating a function with the trapezoidal rule.</i></p>
<!-- end figure -->

Earlier we approximated the area using the midpoint value: $f(x_k+h/2)\cdot h$. Now we use $A=A_1+A_2$, where $A_1=f(x_k)\cdot h$ 
and $A_2=(f(x_k+h)-f(x_k))\cdot h/2$, hence the area of one trapezoid is:

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
A\equiv T(x_k,x_k+h)=(f(x_k+h)+f(x_k))h/2.
\label{_auto1} \tag{3}
\end{equation}
$$

This is the trapezoidal rule, and for the whole interval we get:

$$
I(a,b)=\int_a^bf(x)dx\simeq\frac{1}{2}h\sum_{k=0}^{N-1}\left[f(x_k+h)+f(x_k)\right] \nonumber
$$

$$
=h\left[\frac{1}{2}f(a)+f(a+h) + f(a+2h) +\nonumber\right.
$$

$$
\left.\qquad\cdots + f(a+(N-2)h)+\frac{1}{2}f(b)\right]\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}  
=h\left[\frac{1}{2}f(a)+\frac{1}{2}f(b)+\sum_{k=1}^{N-2}f(a+k h)\right].
\label{_auto2} \tag{4}
\end{equation}
$$

Note that this formula was bit more involved to derive, but it requires only one more function evaluations compared to the midpoint rule. 
Below is a python implementation:

In [3]:
def trapezoidal(f,a,b,N):
    """
    f : function to be integrated on the domain [a,b]
    N : number of integration points
    """
    h=(b-a)/N
    x=np.arange(a+h,b,h)
    return h*(0.5*f(a)+0.5*f(b)+np.sum(f(x)))

In the table below, we have calculated the numerical error for various values of $N$.

<table class="table" border="1">
<thead>
<tr><th align="center">$N$</th> <th align="center"> $h$ </th> <th align="center">Error Midpoint</th> <th align="center">Error Trapezoidal</th> </tr>
</thead>
<tbody>
<tr><td align="center">   1      </td> <td align="center">   3.14     </td> <td align="center">   -57\%             </td> <td align="center">   100\%                </td> </tr>
<tr><td align="center">   5      </td> <td align="center">   0.628    </td> <td align="center">   -1.66\%           </td> <td align="center">   3.31\%               </td> </tr>
<tr><td align="center">   10     </td> <td align="center">   0.314    </td> <td align="center">   -0.412\%          </td> <td align="center">   0.824\%              </td> </tr>
<tr><td align="center">   100    </td> <td align="center">   0.031    </td> <td align="center">   -4.11E-3\%        </td> <td align="center">   8.22E-3\%            </td> </tr>
</tbody>
</table>

Note that we get the surprising result that this algorithm performs poorer, a factor of 2 than the midpoint rule.
How can this be explained? By just looking at [figure 1](#fig:numint:mid), we see that the midpoint rule actually over predicts the area from $[x_k,x_k+h/2]$ 
 and under predicts in the interval $[x_k+h/2,x_{k+1}]$ or vice versa. The net effect is that for many cases the midpoint rule give a slightly better 
 performance than the trapezoidal rule. In the next section we will investigate this more formally.

## Numerical errors on integrals
It is important to know the accuracy of the methods we are using, otherwise we do not know if the
computer produce correct results. In the previous examples we were able to estimate the error because we knew the analytical result. However, if we know the 
analytical result there is no reason to use the computer to calculate the result(!). Thus, we need a general method to estimate the error, and let the computer 
run until a desired accuracy is reached. 

In order to analyze the midpoint rule in more detail we approximate the function by a Taylor 
series at the midpoint between $x_k$ and $x_k+h$:

$$
f(x)=f(x_k+h/2)+f^\prime(x_k+h/2)(x-(x_k+h/2))\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}  
+\frac{1}{2!}f^{\prime\prime}(x_k+h/2)(x-(x_k+h/2))^2+\mathcal{O}(h^3)
\label{_auto3} \tag{5}
\end{equation}
$$

Since $f(x_k+h/2)$ and its derivatives are constants it is straight forward to integrate $f(x)$:

$$
I(x_k,x_k+h)=\int_{x_k}^{x_k+h}\left[f(x_k+h/2)+f^\prime(x_k+h/2)(x-(x_k+h/2))\right.\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}  
\left.+\frac{1}{2!}f^{\prime\prime}(x_k+h/2)(x-(x_k+h/2))^2+\mathcal{O}(h^3)\right]dx
\label{_auto4} \tag{6}
\end{equation}
$$

The first term is simply the midpoint rule, to evaluate the two other terms we make the substitution: $u=x-x_k$:

$$
I(x_k,x_k+h)=f(x_k+h/2)\cdot h+f^\prime(x_k+h/2)\int_0^h(u-h/2)du\nonumber
$$

$$
+\frac{1}{2}f^{\prime\prime}(x_k+h/2)\int_0^h(u-h/2)^2du+\mathcal{O}(h^4)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}  
=f(x_k+h/2)\cdot h-\frac{h^3}{24}f^{\prime\prime}(x_k+h/2)+\mathcal{O}(h^4).
\label{_auto5} \tag{7}
\end{equation}
$$

Note that all the odd terms cancels out, i.e $\int_0^h(u-h/2)^m=0$ for $m=1,3,5\ldots$. Thus the error for the midpoint rule, $E_{M,k}$, on this particular interval is:

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}
E_{M,k}=I(x_k,x_k+h)-f(x_k+h/2)\cdot h=-\frac{h^3}{24}f^{\prime\prime}(x_k+h/2),
\label{_auto6} \tag{8}
\end{equation}
$$

where we have ignored higher order terms. We can easily sum up the error on all the intervals, but clearly $f^{\prime\prime}(x_k+h/2)$ will 
not, in general, have the same value on all intervals. However, an upper bound for the error can be found by replacing $f^{\prime\prime}(x_k+h/2)$ 
with the maximal value on the interval $[a,b]$, $f^{\prime\prime}(\eta)$:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:em"></div>

$$
\begin{equation}
E_{M}=\sum_{k=0}^{N-1}E_{M,k}=-\frac{h^3}{24}\sum_{k=0}^{N-1}f^{\prime\prime}(x_k+h/2)\leq-\frac{Nh^3}{24}f^{\prime\prime}(\eta),\label{eq:numint:em} \tag{9}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation}  
E_{M}\leq-\frac{(b-a)^3}{24N^2}f^{\prime\prime}(\eta),
\label{_auto7} \tag{10}
\end{equation}
$$

where we have used $h=(b-a)/N$. We can do the exact same analysis for the trapezoidal rule, but then we expand the function around $x_k-h$ instead of the midpoint. 
The error term is then:

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation}
E_T=\frac{(b-a)^3}{12N^2}f^{\prime\prime}(\overline{\eta}).
\label{_auto8} \tag{11}
\end{equation}
$$

At the first glance it might look like the midpoint rule always is better than the trapezoidal rule, but note that the second derivative is 
evaluated in different points ($\eta$ and $\overline{\eta}$). Thus it is possible to construct examples where the midpoint rule performs poorer 
than the trapezoidal rule.

Before we end this section we will rewrite the error terms in a more useful form as it is not so easy to evaluate 
$f^{\prime\prime}(\eta)$ (since we do not know which value of $\eta$ to use). By taking a closer look at equation ([9](#eq:numint:em)), 
we see that it is closely related to the midpoint rule for $\int_a^bf^{\prime\prime}(x)dx$, hence:

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation}
E_{M}=-\frac{h^2}{24}h
\sum_{k=0}^{N-1}f^{\prime\prime}(x_k+h/2)\simeq-\frac{h^2}{24}\int_a^b
f^{\prime\prime}(x)dx
\label{_auto9} \tag{12}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation}  
E_M\simeq\frac{h^2}{24}\left[f^\prime(b)-f^\prime(a)\right]=-\frac{(b-a)^2}{24N^2}\left[f^\prime(b)-f^\prime(a)\right]
\label{_auto10} \tag{13}
\end{equation}
$$

The corresponding formula for the trapezoid formula is:

<!-- Equation labels as ordinary links -->
<div id="_auto11"></div>

$$
\begin{equation}
E_T\simeq \frac{h^2}{12}\left[f^\prime(b)-f^\prime(a)\right]=\frac{(b-a)^2}{12N^2}\left[f^\prime(b)-f^\prime(a)\right]
\label{_auto11} \tag{14}
\end{equation}
$$

## Practical estimation of errors on integrals (Richardson extrapolation)
<div id="sec:numint:parct"></div>
From the example above we were able to estimate the number of steps needed to reach (at least) a certain precision. 
In many practical cases we do not deal with functions, but with data and it can be difficult to evaluate the derivative. 
We also saw from the example above that the algorithm gives a higher precision than what we asked for. 
How can we avoid doing too many iterations? A very simple solution to this question is to double the number of intervals until 
a desired accuracy is reached. The following analysis holds for both the trapezoid and midpoint method, because in both cases 
the (global) error scale as $h^2$[^trapez].
[^trapez]: You can do the following analysis by assuming that the local error is $h^3$, but then you need to take into account that you need to take twice as many steps, which will give the same result.

Assume that we have evaluated the integral with a step size $h_1$, and the computed result is $I_1$. 
Then we know that the true integral is $I=I_1+c h_1^2$, where $c$ is a constant that is unknown. If we now half the step size: $h_2=h_1/2$, 
then we get a new (better) estimate of the integral, $I_2$, which is related to the true integral $I$ as: $I=I_2+c h_2^2$. 
Taking the difference between $I_2$ and $I_1$ give us an estimation of the error:

<!-- Equation labels as ordinary links -->
<div id="_auto12"></div>

$$
\begin{equation}
I_2-I_1=I-c h_2^2-(I-ch_1^2)=3c h_2^2,
\label{_auto12} \tag{15}
\end{equation}
$$

where we have used the fact that $h_1=2h_2$, Thus the error term is:

<!-- Equation labels as ordinary links -->
<div id="_auto13"></div>

$$
\begin{equation}
E(a,b)=c h_2^2=\frac{1}{3}(I_2-I_1).
\label{_auto13} \tag{16}
\end{equation}
$$

This might seem like we need to evaluate the integral twice as many times as needed. This is not the case, by choosing to exactly 
half the spacing we only need to evaluate for the values that lies halfway between the original points. We will demonstrate how 
to do this by using the trapezoidal rule, because it operates directly on the $x_k$ values and not the midpoint values. 
The trapezoidal rule can now be written as:

<!-- Equation labels as ordinary links -->
<div id="_auto14"></div>

$$
\begin{equation}
I_2(a,b)=h_2\left[\frac{1}{2}f(a)+\frac{1}{2}f(b)+\sum_{k=1}^{N_2-1}f(a+k h_2)\right],
\label{_auto14} \tag{17}
\end{equation}
$$

$$
=h_2\left[\frac{1}{2}f(a)+\frac{1}{2}f(b)+\sum_{k=\text{even values}}^{N_2-1}f(a+k h_2)\right.\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto15"></div>

$$
\begin{equation}  
\left.\qquad+\sum_{k=\text{odd values}}^{N_2-1}f(a+k h_2)\right],
\label{_auto15} \tag{18}
\end{equation}
$$

in the last equation we have split the sum into odd an even values. The sum over the even values can be rewritten:

<!-- Equation labels as ordinary links -->
<div id="_auto16"></div>

$$
\begin{equation}
\sum_{k=\text{even values}}^{N_2-1}f(a+k h_2)=\sum_{k=0}^{N_1-1}f(a+2k h_2)=\sum_{k=0}^{N_1-1}f(a+k h_1),
\label{_auto16} \tag{19}
\end{equation}
$$

note that $N_2$ is replaced with $N_1=N_2/2$, we can now rewrite $I_2$ as:

$$
I_2(a,b)=h_2\left[\frac{1}{2}f(a)+\frac{1}{2}f(b)+\sum_{k=0}^{N_1-1}f(a+k h_1)\right.\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto17"></div>

$$
\begin{equation}  
\left.+\sum_{k=\text{odd values}}^{N_2-1}f(a+k h_2)\right]
\label{_auto17} \tag{20}
\end{equation}
$$

Note that the first terms are actually the trapezoidal rule for $I_1$, hence:

<!-- Equation labels as ordinary links -->
<div id="_auto18"></div>

$$
\begin{equation}
I_2(a,b)=\frac{1}{2}I_1(a,b)+h_2\sum_{k=\text{odd values}}^{N_2-1}f(a+k h_2).
\label{_auto18} \tag{21}
\end{equation}
$$

The factor $1/2$ in front of $I_1(a,b)$, appears because $h_2=h_1/2$. 
A possible algorithm is then:
1. Choose a low number of steps to evaluate the integral, $I_0$, the first time, e.g. $N_0=1$

2. Double the number of steps, $N_1=2N_0$ 

3. Calculate the missing values by summing over the odd number of steps $\sum_{k=\text{odd values}}^{N_1-1}f(a+k h_1)$

4. Check if $E_1(a,b)=\frac{1}{3}(I_1-I_0)$ is lower than a specific tolerance

5. If yes quit, if not, return to 2, and continue until $E_i(a,b)=\frac{1}{3}(I_{i+1}-I_{i})$ is lower than the tolerance  

Below is a Python implementation:

In [4]:
def int_adaptive_trapez2(lower_limit, upper_limit,func,tol):
    """
    adaptive quadrature, integrate a function from lower_limit
    to upper_limit within tol*(upper_limit-lower_limit)

    """
    S=[]
    S.append([lower_limit,upper_limit])
    I=0
    iterations=0
    while S:
        iterations +=1
        a,b=S.pop(-1) # last element
        m=(b+a)*0.5   # midpoint
        I1=0.5*(b-a)*(func(a)+func(b)) #trapezoidal for 1 interval 
        I2=0.25*(b-a)*(func(a)+func(b)+2*func(m)) #trapezoidal for 2 intervals
        if(np.abs(I1-I2)<3*np.abs((b-a)*tol)):
            I+=I2     # accuarcy met
        else:
            S.append([a,m]) # half the interval 
            S.append([m,b])
    print("Number of iterations: ", iterations)
    return I

What is a good number to start with, what happens if we choose $N_0$ too large? Compare the adaptive midpoint rule with the adaptive 
trapezoidal rule, is it possible to get the same accuracy with the same number of iterations? Check the expected number of 
iterations with the theoretical value $N=\sqrt{\frac{(b-a)^2}{12E_T}\left[f^\prime(b)-f^\prime(a)\right]}$.

If you compare the number of terms used in the adaptive trapezoidal rule, which was developed by halving the step size, and the adaptive midpoint rule that was derived on the basis of the theoretical error term, you will find the adaptive midpoint rule is more efficient. So why go through all this trouble? In the next section we will see that the development we did for the adaptive trapezoidal rule is closely related to Romberg integration, which is *much* more effective.

# Romberg integration
The adaptive algorithm for the trapezoidal rule in the previous section can be easily improved by remembering 
that the true integral was given by[^romerr] : $I=I_i+ch_i^2+\mathcal{O}(h^4)$. The error term was in the previous example only used to 
check if the desired tolerance was achieved, but we could also have added it to our estimate of the integral to reach an accuracy to fourth order:

[^romerr]: Note that all odd powers of $h$ is equal to zero, thus the corrections are always in even powers.

<!-- Equation labels as ordinary links -->
<div id="_auto19"></div>

$$
\begin{equation}
I=I_{i+1}+ch^2+\mathcal{O}(h^4)=I_{i+1}+\frac{1}{3}\left[I_{i+1}-I_{i}\right]+\mathcal{O}(h^4).
\label{_auto19} \tag{22}
\end{equation}
$$

As before the error term $\mathcal{O}(h^4)$, can be written as: $ch^4$. Now we can proceed as in the previous section: First we estimate the 
integral by one step size $I_i=I+ch_i^4$, next we half the step size $I_{i+1}=I+ch_{i+1}^4$ and use these two estimates to calculate the error term:

$$
I_{i+1}-I_{i}=I-c h_{i+1}^4-(I-ch_i^4)=-c h_{i+1}^4+c(2h_{i+1})^4=15c h_{i+1}^4,\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto20"></div>

$$
\begin{equation}  
ch_{i+1}^4=\frac{1}{15}\left[I_{i+1}-I_{i}\right]+\mathcal{O}(h^6).
\label{_auto20} \tag{23}
\end{equation}
$$

but now we are in the exact situation as before, we have not only the error term but the correction up to order $h^4$ for this integral:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:rom"></div>

$$
\begin{equation}
I=I_{i+1}+\frac{1}{15}\left[I_{i+1}-I_{i}\right]+\mathcal{O}(h^6).\label{eq:numint:rom} \tag{24}
\end{equation}
$$

Each time we half the step size we also gain a higher order accuracy in our numerical algorithm. Thus, there are two iterations going on at the same time; 
one is the iteration that half the step size ($i$), and the other one is the increasing number of higher order terms added (which we will denote $m$). 
We need to improve our notation, and replace the approximation of the integral ($I_i$) with $R_{i,m}$. Equation ([24](#eq:numint:rom)), can now 
be written:

<!-- Equation labels as ordinary links -->
<div id="_auto21"></div>

$$
\begin{equation}
I=R_{i+1,2}+\frac{1}{15}\left[R_{i+1,2}-R_{i,2}\right]+\mathcal{O}(h^6).
\label{_auto21} \tag{25}
\end{equation}
$$

A general formula valid for any $m$ can be found by realizing:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:rom0"></div>

$$
\begin{equation}
I=R_{i+1,m+1}+c_mh_i^{2m+2}+\mathcal{O}(h_i^{2m+4})\label{eq:numint:rom0} \tag{26}
\end{equation}
$$

$$
I=R_{i,m+1}+c_mh_{i-1}^{2m+2}+\mathcal{O}(h_{i-1}^{2m+4})\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:numint:rom1"></div>

$$
\begin{equation}  
=R_{i,m+1}+2^{2m+2}c_mh_{i}^{2m+2}+\mathcal{O}(h_{i-1}^{2m+4}),\label{eq:numint:rom1} \tag{27}
\end{equation}
$$

where, as before $h_{i-1}=2h_i$. Subtracting equation ([26](#eq:numint:rom0)) and ([27](#eq:numint:rom1)), we find an expression for the error term:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:rom2"></div>

$$
\begin{equation}
c_mh_{i}^{2m+2}=\frac{1}{4^{m+1}-1}(R_{i,m}-R_{i-1,m})\label{eq:numint:rom2} \tag{28}
\end{equation}
$$

Then the estimate for the integral in equation ([27](#eq:numint:rom1)) is:

<!-- Equation labels as ordinary links -->
<div id="_auto22"></div>

$$
\begin{equation}
I=R_{i,m+1}+\mathcal{O}(h_i^{2m+2})
\label{_auto22} \tag{29}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto23"></div>

$$
\begin{equation}  
R_{i,m+1}=R_{i,m}+\frac{1}{4^{m+1}-1}(R_{i+1,m}-R_{i,m}).
\label{_auto23} \tag{30}
\end{equation}
$$

A possible algorithm is then:

1. Evaluate $R_{0,0}=\frac{1}{2}\left[f(a)+f(b)\right](b-a)$ as the first estimate

2. Double the number of steps, $N_{i+1}=2N_i$ or half the step size $h_{i+1}=h_i/2$ 

3. Calculate the missing values by summing over the odd number of steps $\sum_{k=\text{odd values}}^{N_1-1}f(a+k h_{i+1})$

4. Correct the estimate by adding *all* the higher order error term $R_{i,m+1}=R_{i,m}+\frac{1}{4^m-1}(R_{i+1,m+1}-R_{i,m+1})$

5. Check if the error term is lower than a specific tolerance $E_{i,m}(a,b)=\frac{1}{4^{m+1}-1}(R_{i,m}-R_{i-1,m})$, if yes quit, if no goto 2, increase $i$ and $m$ by one

The algorithm is illustrated in [figure 3](#fig:numint:romberg).
<!-- dom:FIGURE: [fig-numint/romberg.png, width=800 frac=0.5] Illustration of the Romberg algorithm. Note that for each new evaluation of the integral $R_{i,0}$, all the correction terms $R_{i,m}$ (for $m>0$) must be evaluated again. <div id="fig:numint:romberg"></div> -->
<!-- begin figure -->
<div id="fig:numint:romberg"></div>

<img src="fig-numint/romberg.png" width=800><p style="font-size: 0.9em"><i>Figure 3: Illustration of the Romberg algorithm. Note that for each new evaluation of the integral $R_{i,0}$, all the correction terms $R_{i,m}$ (for $m>0$) must be evaluated again.</i></p>
<!-- end figure -->

Note that the tolerance term is not the correct one as it uses the error estimate for the current step, 
which we also use correct the integral in the current step to reach a higher accuracy. 
Thus the error on the integral will always be lower than the user specified tolerance.
Below is a Python implementation:

In [5]:
def int_romberg(func,a, b,tol,show=False):
    """ calculates the area of func on the domain [a,b]
        for the given tol, if show=True the triangular
        array of intermediate results are printed """
    Nmax = 100
    R = np.empty([Nmax,Nmax]) # storage buffer
    h = (b-a) # step size
    R[0,0]    =.5*(func(a)+func(b))*h
    N = 1
    for i in range(1,Nmax):
        h /= 2
        N *= 2
        odd_terms=0
        for k in range (1,N,2): # 1, 3, 5, ... , N-1
            val        = a + k*h
            odd_terms += func(val)
		# add the odd terms to the previous estimate	
        R[i,0]   = 0.5*R[i-1,0] + h*odd_terms 
        for m in range(i): 
			# add all higher order terms in h
            R[i,m+1]   = R[i,m] + (R[i,m]-R[i-1,m])/(4**(m+1)-1)                  
		# check tolerance, best guess			
        calc_tol = abs(R[i,i]-R[i-1,i-1])       
        if(calc_tol<tol):
            break  # estimated precision reached 
    if(i == Nmax-1):
        print('Romberg routine did not converge after ',
              Nmax, 'iterations!')
    else:      
        print('Number of intervals = ', N)

    if(show==True):
        elem = [2**idx for idx in range(i+1)]
        print("Steps StepSize Results")
        for idx in range(i+1):
            print(elem[idx],' ',
                  "{:.6f}".format((b-a)/2**idx),end = ' ')
            for l in range(idx+1):
                print("{:.6f}".format(R[idx,l]),end = ' ')
            print('')  
    return R[i,i] #return the best estimate

Note that the Romberg integration only uses 32 function evaluations to reach a precision of $10^{-8}$, whereas the adaptive midpoint and trapezoidal rule in the previous
section uses 20480 and 9069 function evaluations, respectively. 

## Alternative implementation of adaptive integration
Before we proceed, we will consider an alternative implementation of the adaptive method presented in the previous sections, with the following modification
1. We will use Simpsons rule (see the exercise at the end), which takes the following form $\int_a^bf(x)dx\simeq\frac{h}{6}\left[f(a)+4f(a+\frac{h}{2})+2f(a+h)+ 4f(a+3\frac{h}{2})+2f(a+2h)+\cdots+f(b)\right]$

2. We only divide the intervals needed to reach the desired accuracy.

Simpsons rule is accurate up to $\mathcal{O}(h^4)$, and by following the same arguments as above we can estimate the error as $E_i(a,b)=\frac{1}{15}(I_{i+1}-I_{i})$. The factor 1/15 (as opposed to 1/3) originates from the higher order accuracy. The integration proceeds as follows
* `S` is an empty list

* `S.append([a,b])`

* $I=0$

* `while S not empty do:`

  * `[a,b]=S.pop(-1)`

  * $m=(b+a)/2$

  * $I_1=$ `simpson_step(a,b)`

  * $I_2=$ `simpson_step(a,m)+simpson_step(m,b)`

  * if $|I_1-I_2|<15|b-a|\cdot tol$

    * $I+=I_2$


  * else:

    * `S.append([a,m])`

    * `S.append([m,b])`


  * return $I$


Note the use of the list `S`, we remove the interval $[a,b]$ from the list and calculates the integral. If the integral is not accurate enough we add to new intervals to the list, and continue until we reach the desired accuracy, then we proceed with the next interval. Since we remove (`pop`) the element from the list, we know that we will finish the evaluation once the list is empty. This algorithm allows for different sub interval to have different degrees of subdivisions, contrary to Rombergs algorithm. The full python implementation is shown below

In [6]:
def simpson_step(a, b,func):
    m=0.5*(a+b)
    return (b-a)/6*(func(a)+func(b)+4*func(m))

def int_adaptive_simpson(func,a, b,tol):
    """
    adaptive quadrature, integrate a function from a
    to b within tol*(b-a) uses simpsons rule
    """
    S=[]
    S.append([a,b])
    I=0
    iterations=0
    while S:
        iterations +=1
        a,b=S.pop(-1) # last element
        m=(b+a)*0.5   # midpoint
        I1=simpson_step(a,b,func) #simpsons for 1 interval 
        I2=simpson_step(a,m,func)+simpson_step(m,b,func) # ...2 intervals
        if(np.abs(I1-I2)<15*np.abs((b-a)*tol)):
            I+=I2     # accuarcy met
        else:
            S.append([a,m]) # half the interval 
            S.append([m,b])
    print("Number of iterations: ", iterations)
    return I

# Gaussian quadrature
Many of the methods we have looked into are of the type:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:qq1"></div>

$$
\begin{equation}
	\int_a^b f(x) dx = \sum_{k=0}^{N-1} \omega_k f(x_k),\label{eq:numint:qq1} \tag{31}
\end{equation}
$$

where the function is evaluated at fixed interval. For the midpoint rule $\omega_k=h$ for all values of $k$, for the trapezoid rule 
$\omega_k=h/2$ for the endpoints and $h$ for all the interior points. 
For the Simpsons rule (see exercise) $\omega_k=h/3, 4h/3,2h/3,4h/3,\ldots,4h/3,h/3$. 
Note that all the methods we have looked at so far samples the function in equal spaced points, $f(a+k h)$, 
for $k=0, 1, 2\ldots, N-1$. If we now allow for the function to be evaluated at unevenly spaced points, we can do a lot better. 
This realization is the basis for Gaussian Quadrature. We will explore this in the following, 
but to make the development easier and less cumbersome, we transform the integral from the domain $[a,b]$ to $[-1,1]$:

<!-- Equation labels as ordinary links -->
<div id="_auto24"></div>

$$
\begin{equation}
\int_a^bf(t)dt=\frac{b-a}{2}\int_{-1}^{1}f(x)dx\text{ , where:}
\label{_auto24} \tag{32}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto25"></div>

$$
\begin{equation}  
x=\frac{2}{b-a}t-\frac{b+a}{b-a}.
\label{_auto25} \tag{33}
\end{equation}
$$

The factor in front comes from the fact that $dt=(b-a)dx/2$, thus we can develop our algorithms on the domain $[-1,1]$, 
and then do the transformation back using: $t=(b-a)x/2+(b+a)/2$.

**Notice.**

The idea we will explore is as follows:
If we can approximate the function to be integrated on the domain $[-1,1]$ (or on $[a,b]$) as a 
polynomial of as *large a degree as possible*, then the numerical integral of this polynomial will be very close to the integral of the 
function we are seeking.


This idea is best understood by a couple of examples. Assume that we want to use $N=1$ in equation ([31](#eq:numint:qq1)):

<!-- Equation labels as ordinary links -->
<div id="_auto26"></div>

$$
\begin{equation}
\int_{-1}^{1}f(x)\,dx\simeq\omega_0f(x_0).
\label{_auto26} \tag{34}
\end{equation}
$$

We now choose $f(x)$ to be a polynomial of as large a degree as possible, but with the requirement that the integral is exact. If $f(x)=1$, we get:

<!-- Equation labels as ordinary links -->
<div id="_auto27"></div>

$$
\begin{equation}
\int_{-1}^{1}f(x)\,dx=\int_{-1}^{1}1\,dx=2=\omega_0,
\label{_auto27} \tag{35}
\end{equation}
$$

hence $\omega_0=2$. If we choose $f(x)=x$, we get:

<!-- Equation labels as ordinary links -->
<div id="_auto28"></div>

$$
\begin{equation}
\int_{-1}^{1}f(x)\,dx=\int_{-1}^{1}x\,dx=0=\omega_0f(x_0)=2x_0,
\label{_auto28} \tag{36}
\end{equation}
$$

hence $x_0=0$. 
**The Gaussian integration rule for $N=1$ is:**

$$
\int_{-1}^{1}f(x)\,dx\simeq 2f(0)\text{, or: }\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto29"></div>

$$
\begin{equation}  
\int_{a}^{b}f(t)\,dt\simeq\frac{b-a}{2}\,2f(\frac{b+a}{2})=(b-a)f(\frac{b+a}{2}).
\label{_auto29} \tag{37}
\end{equation}
$$

This equation is equal to the midpoint rule, by choosing $b=a+h$ we reproduce equation ([1](#eq:numint:mid0)). If we choose $N=2$:

<!-- Equation labels as ordinary links -->
<div id="_auto30"></div>

$$
\begin{equation}
\int_{-1}^{1}f(x)\,dx\simeq\omega_0f(x_0)+\omega_1f(x_1),
\label{_auto30} \tag{38}
\end{equation}
$$

we can show that now $ f(x)=1,\,x,\,x^2\,x^3$ can be integrated exact:

<!-- Equation labels as ordinary links -->
<div id="_auto31"></div>

$$
\begin{equation}
\int_{-1}^{1}1\,dx=2=\omega_0f(x_0)+\omega_1f(x_1)=\omega_0+\omega_1\,,
\label{_auto31} \tag{39}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto32"></div>

$$
\begin{equation}  
\int_{-1}^{1}x\,dx=0=\omega_0f(x_0)+\omega_1f(x_1)=\omega_0x_0+\omega_1x_1\,,
\label{_auto32} \tag{40}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto33"></div>

$$
\begin{equation}  
\int_{-1}^{1}x^2\,dx=\frac{2}{3}=\omega_0f(x_0)+\omega_1f(x_1)=\omega_0x_0^2+\omega_1x_1^2\,,
\label{_auto33} \tag{41}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto34"></div>

$$
\begin{equation}  
\int_{-1}^{1}x^3\,dx=0=\omega_0f(x_0)+\omega_1f(x_1)=\omega_0x_0^3+\omega_1x_1^3\,,
\label{_auto34} \tag{42}
\end{equation}
$$

hence there are four unknowns and four equations. The solution is: $\omega_0=\omega_1=1$ and $x_0=-x_1=1/\sqrt{3}$.

**The Gaussian integration rule for $N=2$ is:**

<!-- Equation labels as ordinary links -->
<div id="_auto35"></div>

$$
\begin{equation}
\int_{-1}^{1}f(x)\,dx\simeq f(-\frac{1}{\sqrt{3}})+f(\frac{1}{\sqrt{3}})\, \text{, or:}
\label{_auto35} \tag{43}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto36"></div>

$$
\begin{equation}  
\int_{a}^{b}f(x)\,dx\simeq \frac{b-a}{2}\left[f(-\frac{b-a}{2}\frac{1}{\sqrt{3}}+\frac{b+a}{2})
+f(\frac{b-a}{2}\frac{1}{\sqrt{3}}+\frac{b+a}{2})\right].
\label{_auto36} \tag{44}
\end{equation}
$$

In [7]:
def int_gaussquad2(func, lower_limit, upper_limit):
    x   = np.array([-1/np.sqrt(3.),1/np.sqrt(3)])
    w   = np.array([1, 1])
    xp  = 0.5*(upper_limit-lower_limit)*x
    xp += 0.5*(upper_limit+lower_limit)
    area = np.sum(w*func(xp))
    return area*0.5*(upper_limit-lower_limit)

### The case N=3

For the case $N=3$, we find that $f(x)=1,x,x^2,x^3,x^4,x^5$ can be integrated exactly:

<!-- Equation labels as ordinary links -->
<div id="_auto37"></div>

$$
\begin{equation}
\int_{-1}^{1}1\,dx=2=\omega_0+\omega_1+\omega_2\,,
\label{_auto37} \tag{45}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto38"></div>

$$
\begin{equation}  
\int_{-1}^{1}x\,dx=0=\omega_0x_0+\omega_1x_1+\omega_2x_2\,,
\label{_auto38} \tag{46}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto39"></div>

$$
\begin{equation}  
\int_{-1}^{1}x^2\,dx=\frac{2}{3}=\omega_0x_0^2+\omega_1x_1^2+\omega_2x_2^2\,,
\label{_auto39} \tag{47}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto40"></div>

$$
\begin{equation}  
\int_{-1}^{1}x^3\,dx=0=\omega_0x_0^3+\omega_1x_1^3+\omega_2x_2^3\,,
\label{_auto40} \tag{48}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto41"></div>

$$
\begin{equation}  
\int_{-1}^{1}x^4\,dx=\frac{2}{5}=\omega_0x_0^4+\omega_1x_1^4+\omega_2x_2^4\,,
\label{_auto41} \tag{49}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto42"></div>

$$
\begin{equation}  
\int_{-1}^{1}x^5\,dx=0=\omega_0x_0^5+\omega_1x_1^5+\omega_2x_2^5\,,
\label{_auto42} \tag{50}
\end{equation}
$$

the solution to these equations are $\omega_{0,1,2}=5/9, 8/9, 5/9$ and $x_{1,2,3}=-\sqrt{3/5},0,\sqrt{3/5}$. Below is a Python implementation:

In [8]:
def int_gaussquad3(lower_limit, upper_limit,func):
    x  = np.array([-np.sqrt(3./5.),0.,np.sqrt(3./5.)])
    w  = np.array([5./9., 8./9., 5./9.])
    xp = 0.5*(upper_limit-lower_limit)*x
    xp += 0.5*(upper_limit+lower_limit)
    area = np.sum(w*func(xp))
    return area*0.5*(upper_limit-lower_limit)

Note that the Gaussian quadrature converges very fast. From $N=2$ to $N=3$ function evaluation we reduce the error (in this specific case) 
from 6.5% to 0.1%. Our standard trapezoidal formula needs more than 20 function evaluations to achieve this, the Romberg method uses 4-5 function
evaluations. How can this be? If we use the standard Taylor formula for the function to be integrated, we know that for $N=2$ the Taylor 
formula must be integrated up to $x^3$, so the error term is proportional to $h^4f^{(4)}(\xi)$ (where $\xi$ is some x-value in $[a,b]$). 
$h$ is the step size, and we can replace it with $h\sim (b-a)/N$, thus the error scale as $c_N/N^4$ (where $c_N$ is a constant). 
Following the same argument, we find for $N=3$ that the error term is $h^6f^{(6)}(\xi)$ or that the error term scale as $c_N/N^6$. 
Each time we increase $N$ by a factor of one, the error term reduces by $N^2$. Thus if we evaluate the integral for $N=10$, 
increasing to $N=11$ will reduce the error by a factor of $11^2=121$.

## Error term on Gaussian integration
The Gaussian integration rule of order $N$ integrates exactly a polynomial of order $2N-1$. 
From Taylors error formula, 
we can easily see that the error term must be of order $2N$, and be proportional to $f^{(2N)}(\eta)$, see [[stoer2013]](#stoer2013) for more details on the derivation of error terms. The drawback with an analytical error term derived from series expansion is that it involves the derivative of the function. As we have already explained, this is very unpractical and it is much more practical to use the methods described in the section [sec:numint:parct](#sec:numint:parct). Let us consider this in more detail, assume that we evaluate the integral using first a Gaussian integration rule with $N$ points, and then $N+1$ points. Our estimates of the "exact" integral, $I$,  would then be:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:gerr1"></div>

$$
\begin{equation}
 I=I_N+ch_{N}^{2N},\label{eq:numint:gerr1} \tag{51}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:numint:gerr2"></div>

$$
\begin{equation}  
 I=I_{N+1}+ch_{N+1}^{2N+1}.
\label{eq:numint:gerr2} \tag{52}
\end{equation}
$$

In principle $h_{N+1}\neq h_{N}$, but in the following we will assume that $h_N\simeq h_{N+1}$, and $h\ll 1$. Subtracting equation ([51](#eq:numint:gerr1)) and ([52](#eq:numint:gerr2)) we can show that a reasonable estimate for the error term $ch^{2N}$ would be:

<!-- Equation labels as ordinary links -->
<div id="_auto43"></div>

$$
\begin{equation}
ch^N= I_{N+1}-I_N.
\label{_auto43} \tag{53}
\end{equation}
$$

If this estimate is lower than a given tolerance we can be quite confident that the higher order estimate $I_{N+1}$ approximate the true integral within our error estimate. This is the method implemented in SciPy, [`integrate.quadrature`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.quadrature.html)

## Common weight functions for classical Gaussian quadratures
# Integrating functions over an infinite range
Integrating a function over an infinite range can be done by the following trick. Assume that we would like to evaluate

<!-- Equation labels as ordinary links -->
<div id="eq:numint:inf"></div>

$$
\begin{equation}
\int_a^\infty f(x) dx.
\label{eq:numint:inf} \tag{54}
\end{equation}
$$

If we introduce the following substitution

<!-- Equation labels as ordinary links -->
<div id="eq:numint:infs"></div> <div id="eq:numint:infs2"></div>

$$
\begin{equation}
z=\frac{x-a}{1+x-a},
\label{eq:numint:infs} \tag{55}
\end{equation}
or equivalently
\begin{equation}
x=a+\frac{z}{1-z},
\label{eq:numint:infs2} \tag{56}
\end{equation}
$$

then if $x=a$, $z=0$, and if $x\to\infty$ then $z\to1$, hence:

<!-- Equation labels as ordinary links -->
<div id="eq:numint:infs3"></div>

$$
\begin{equation}
\int_a^\infty f(x) dx = \int_0^1 f(a+\frac{z}{1-z}) \frac{dz}{(1-z)^2}.
\label{eq:numint:infs3} \tag{57}
\end{equation}
$$

<!-- --- begin exercise --- -->

## Exercise 1: Numerical Integration

**a)**
Show that for a linear function, $y=a\cdot x+b$ both the trapezoidal rule and the rectangular rule are exact

**b)**
Consider $I(a,b)=\int_a^bf(x)dx$ for $f(x)=x^2$. The analytical result is $I(a,b)=\frac{b^3-a^3}{3}$. Use the Trapezoidal and 
  Midpoint rule to evaluate these integrals and show that the error for the Trapezoidal rule is exactly twice as big as the Midpoint rule.

**c)**
Use the fact that the error term on the trapezoidal rule is twice as big as the midpoint rule to derive Simpsons formula: $I(a,b)=\sum_{k=0}^{N-1}I(x_k,x_k+h)=\frac{h}{6}\left[f(a)+ 4f(a+\frac{h}{2})+2f(a+h)+4f(a+3\frac{h}{2})+2f(a+2h)+\cdots+f(b)\right]$ Hint: $I(x_k,x_k+h)=M(x_k,x_k+h)+E_M$ (midpoint rule) and $I(x_k,x_k+h)=T(x_k,x_k+h)+E_T=T(x_k,x_k+h)-2E_M$ (trapezoidal rule).

<!-- --- begin solution of exercise --- -->
**Solution.**
Simpsons rule is an improvement over the midpoint and trapezoidal rule. It can be derived in different ways, we will make use of 
the results in the previous section. If we assume that the second derivative is reasonably well behaved on the interval $x_k$ 
and $x_k+h$ and fairly constant we can assume that $f^{\prime\prime}(\eta)\simeq f^{\prime\prime}(\overline{\eta})$, hence $E_T=-2E_M$.

<!-- Equation labels as ordinary links -->
<div id="_auto44"></div>

$$
\begin{equation}
I(x_k,x_k+h)=M(x_k,x_k+h)+E_M\text{ (midpoint rule)}
\label{_auto44} \tag{58}
\end{equation}
$$

$$
I(x_k,x_k+h)=T(x_k,x_k+h)+E_T\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto45"></div>

$$
\begin{equation}  
=T(x_k,x_k+h)-2E_M\text{ (trapezoidal rule)},
\label{_auto45} \tag{59}
\end{equation}
$$

we can now cancel out the error term by multiplying the first equation with 2 and adding the equations:

<!-- Equation labels as ordinary links -->
<div id="_auto46"></div>

$$
\begin{equation}
3I(x_k,x_k+h)=2M(x_k,x_k+h)+T(x_k,x_k+h)
\label{_auto46} \tag{60}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto47"></div>

$$
\begin{equation}  
=2f(x_k+\frac{h}{2}) h+\left[f(x_k+h)+f(x_k)\right] \frac{h}{2}
\label{_auto47} \tag{61}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto48"></div>

$$
\begin{equation}  
I(x_k,x_k+h)=\frac{h}{6}\left[f(x_k)+4f(x_k+\frac{h}{2})+f(x_k+h)\right].
\label{_auto48} \tag{62}
\end{equation}
$$

Now we can do as we did in the case of the trapezoidal rule, sum over all the elements:

$$
I(a,b)=\sum_{k=0}^{N-1}I(x_k,x_k+h)\nonumber
$$

$$
=\frac{h}{6}\left[f(a)+ 4f(a+\frac{h}{2})+2f(a+h)+4f(a+3\frac{h}{2})\right.\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto49"></div>

$$
\begin{equation}  
\left.\qquad+2f(a+2h)+\cdots+f(b)\right]
\label{_auto49} \tag{63}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto50"></div>

$$
\begin{equation}  
=\frac{h^\prime}{3}\left[f(a)+ f(b) + 4\sum_{k= \text{odd}}^{N-2}f(a+k h^\prime)+2\sum_{k= \text{even}}^{N-2}f(a+k h^\prime)\right],
\label{_auto50} \tag{64}
\end{equation}
$$

note that in the last equation we have changed the step size $h=2h^\prime$.
<!-- --- end solution of exercise --- -->

**d)**
Show that for $N=2$ ($f(x)=1,x,x^3$), the points and Gaussian quadrature rule for $\int_{0}^{1}x^{1/2}f(x)\,dx$
is $\omega_{0,1}=-\sqrt{70}{150} + 1/3, \sqrt{70}{150} + 1/3$
and $x_{0,1}=-2\sqrt{70}{63} + 5/9, 2\sqrt{70}{63} + 5/9$
1. Integrate $\int_0^1x^{1/2}\cos x\,dx$ using the rule derived in the exercise above and compare with the standard Gaussian quadrature rule for ($N=2$, and $N=3$).

**e)**
Make a Python program that uses the Midpoint rule to integrate experimental data that are unevenly spaced and given in the form of two arrays.

<!-- --- end exercise --- -->

# References

1. <div id="stoer2013"></div> **J. Stoer and R. Bulirsch**.  *Introduction to Numerical Analysis*, Springer Science & Business Media, 2013.